# Primitives examples

The examples in this section illustrate some common ways to use primitives. Before running these examples, follow the instructions in [Install and set up.](install-qiskit)

<Admonition type="note">
    These examples all use the primitives from Qiskit Runtime, but you could use the base primitives instead.
</Admonition>

## Estimator examples

Efficiently calculate and interpret expectation values of the quantum operators required for many algorithms with Estimator. Explore uses in molecular modeling, machine learning, and complex optimization problems.

### Run a single experiment

Use Estimator to determine the expectation value of a single circuit-observable pair.

In [1]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
observable = SparsePauliOp("Z" * 50)

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

estimator = Estimator(backend)
job = estimator.run([(isa_circuit, isa_observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.203125
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


### Run multiple experiments in a single job

Use Estimator to determine the expectation values of multiple circuit-observable pairs.

In [2]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]

pubs = []
circuits = [IQP(mat) for mat in mats]
observables = [
    SparsePauliOp("X" * 50),
    SparsePauliOp("Y" * 50),
    SparsePauliOp("Z" * 50),
]

# Get ISA circuits
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

for qc, obs in zip(circuits, observables):
    isa_circuit = pm.run(qc)
    isa_obs = obs.apply_layout(isa_circuit.layout)
    pubs.append((isa_circuit, isa_obs))

estimator = Estimator(backend)
job = estimator.run(pubs)
job_result = job.result()

for idx in range(len(pubs)):
    pub_result = job_result[idx]
    print(f">>> Expectation values for PUB {idx}: {pub_result.data.evs}")
    print(f">>> Standard errors for PUB {idx}: {pub_result.data.stds}")

>>> Expectation values for PUB 0: -0.05204460966542751
>>> Standard errors for PUB 0: 0.2668113375877867
>>> Expectation values for PUB 1: 0.05411499436302142
>>> Standard errors for PUB 1: 0.33785760393082975
>>> Expectation values for PUB 2: -0.3070539419087137
>>> Standard errors for PUB 2: 0.2560868761970456


### Run parameterized circuits

Use Estimator to run three experiments in a single job, leveraging parameter values to increase circuit reusability.

In [3]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Step 1: Map classical inputs to a quantum problem
theta = Parameter("θ")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)

number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
individual_phases = [[ph] for ph in phases]

ZZ = SparsePauliOp.from_list([("ZZ", 1)])
ZX = SparsePauliOp.from_list([("ZX", 1)])
XZ = SparsePauliOp.from_list([("XZ", 1)])
XX = SparsePauliOp.from_list([("XX", 1)])
ops = [ZZ, ZX, XZ, XX]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
chsh_isa_circuit = pm.run(chsh_circuit)
isa_observables = [
    operator.apply_layout(chsh_isa_circuit.layout) for operator in ops
]

# Step 3: Execute using Qiskit primitives.

# Reshape observable array for broadcasting
reshaped_ops = np.fromiter(isa_observables, dtype=object)
reshaped_ops = reshaped_ops.reshape((4, 1))

estimator = Estimator(backend, options={"default_shots": int(1e4)})
job = estimator.run([(chsh_isa_circuit, reshaped_ops, individual_phases)])
# Get results for the first (and only) PUB
pub_result = job.result()[0]
print(f">>> Expectation values: {pub_result.data.evs}")
print(f">>> Standard errors: {pub_result.data.stds}")
print(f">>> Metadata: {pub_result.metadata}")

>>> Expectation values: [[ 1.03754483  0.9874777   0.82931695  0.58514344  0.28807852 -0.02413492
  -0.34122669 -0.63007548 -0.8352223  -0.97823454 -1.02445035 -0.98285612
  -0.83470879 -0.58745423 -0.28063264  0.03928343  0.35971302  0.62750793
   0.8408709   0.98670744  1.03395026]
 [ 0.01232421  0.36279407  0.61287293  0.85755994  0.98824797  1.04113939
   0.96411305  0.81468194  0.58283265  0.29090282 -0.03414834 -0.3271052
  -0.62571065 -0.83753309 -0.99209928 -1.0275314  -0.96591033 -0.81827651
  -0.56203554 -0.30374054  0.02285115]
 [-0.05828327 -0.33763213 -0.62981872 -0.86089775 -1.01803149 -1.03857184
  -0.98876148 -0.8092901  -0.58950827 -0.27061921  0.0372294   0.34584828
   0.61852153  0.86372205  0.98927498  1.04190965  0.98362639  0.82520888
   0.61287293  0.31709177 -0.03594563]
 [ 1.04909878  0.9813156   0.82341159  0.58694073  0.31324046 -0.03389159
  -0.36228056 -0.63931864 -0.85730319 -1.01161262 -1.04961229 -0.99646411
  -0.83291151 -0.59233257 -0.29911896  0.02721

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [4]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    EstimatorV2 as Estimator,
)

n_qubits = 50

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
observable = SparsePauliOp("X" * 50)
another_observable = SparsePauliOp("Y" * 50)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
another_isa_observable = another_observable.apply_layout(
    another_isa_circuit.layout
)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session)

    estimator.options.resilience_level = 1

    job = estimator.run([(isa_circuit, isa_observable)])
    another_job = estimator.run(
        [(another_isa_circuit, another_isa_observable)]
    )
    result = job.result()
    another_result = another_job.result()

    # first job
    print(f" > Expectation value: {result[0].data.evs}")
    print(f" > Metadata: {result[0].metadata}")

    # second job
    print(f" > Another Expectation value: {another_result[0].data.evs}")
    print(f" > More Metadata: {another_result[0].metadata}")

 > Expectation value: 0.31654676258992803
 > Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}
 > Another Expectation value: 0.11309157959434542
 > More Metadata: {'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32}


<span id="sampler-examples"></span>
## Sampler examples

Generate entire error-mitigated quasi-probability distributions sampled from quantum circuit outputs. Leverage Sampler’s capabilities for search and classification algorithms like Grover’s and QVSM.

### Run a single experiment

Use Sampler to return the measurement outcome as bitstrings or counts of a single circuit.

In [5]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = IQP(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

sampler = Sampler(backend)
job = sampler.run([isa_circuit])
result = job.result()

# Get results for the first (and only) PUB
pub_result = result[0]

print(f" > First ten results: {pub_result.data.meas.get_bitstrings()[:10]}")

 > First ten results: ['1100011000110101011001011000001010110001000100011011101010101101110110010010010011001101110110001011111100010000100011011100001', '0000000010000100000110011001101010000001100010000110001000110100101010000001010011011000110010001001000100000011100000100010010', '0111011100111001000110010111111010000001011010100010010010100001010010010100000000000001010000000001100100001101010110001111011', '1111001001111001010110101100100110110010011110001001000001000010101000101001111100000110110000000010000100001011100101000100001', '0001001100110001011110101111011111101010110000001000100100010001011000001010010100010000000000101011000000000000000101000110001', '0101110010010101001110110000000101101010100010110101000010000000110000000000000111000000100000001100100010000001100101001101101', '0101110001110100000110010100111011000001110000000001011001000110001010110001010010000001000000000010001100001001000100001110110', '11110110101001110111101000001000000010100000011010011010010

### Run multiple experiments in a single job

Use Sampler to return the measurement outcome as bitstrings or counts of multiple circuits in one job.

In [6]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng()
mats = [np.real(random_hermitian(n_qubits, seed=rng)) for _ in range(3)]
circuits = [IQP(mat) for mat in mats]
for circuit in circuits:
    circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuits = pm.run(circuits)

sampler = Sampler(backend)
job = sampler.run(isa_circuits)
result = job.result()

for idx, pub_result in enumerate(result):
    print(
        f" > First ten results for pub {idx}: {pub_result.data.meas.get_bitstrings()[:10]}"
    )

 > First ten results for pub 0: ['1101100010110001011100010000111001001010011001000000100001001001010111000010010000011001100000010001111000000000001001011000111', '0101111001010010101001100110011010100010011111000010110111001011111011111010010101100110000110011000001100000001101000101001001', '0001010011010100111010111110010000111111010000000010011001001011101101001011001101000000101001100001000010110000011001000001010', '1000001100011100110000011000010101011110011111000100000101010111000100000110010001010100000000010101001110011000001111011000110', '1100110000001111010100100101001100100101011010011101000010010001100001011001000000100100011001111100010000011000001100000001010', '0100000010100111011011000110011001001111000101011001101011110000011100001110011111100001111100011000100010001000001101001101010', '1000100010100010111100001110101011001111000100001000110100100011001000110101110010011100110100000111000110000100100100000000001', '1101110111010000100000100110101100001111101111000

### Run parameterized circuits

Run several experiments in a single job, leveraging parameter values to increase circuit reusability.

In [7]:
import numpy as np
from qiskit.circuit.library import RealAmplitudes
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

# Step 1: Map classical inputs to a quantum problem
circuit = RealAmplitudes(num_qubits=n_qubits, reps=2)
circuit.measure_all()

# Define three sets of parameters for the circuit
rng = np.random.default_rng(1234)
parameter_values = [
    rng.uniform(-np.pi, np.pi, size=circuit.num_parameters) for _ in range(3)
]

# Step 2: Optimize problem for quantum execution.

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

# Step 3: Execute using Qiskit primitives.
sampler = Sampler(backend)
job = sampler.run([(isa_circuit, parameter_values)])
result = job.result()
# Get results for the first (and only) PUB
pub_result = result[0]
# Get counts from the classical register "meas".
print(
    f" >> First ten results for the meas output register: {pub_result.data.meas.get_bitstrings()[:10]}"
)

 >> First ten results for the meas output register: ['1010111011011000100111001100001100100001111110001100110000011010111011111111100100101110111011110101011101011100110010110001011', '0001111011001001100111110100000111010001001010000111100110111101010110111110001110101011010000000100100100010101001001001011000', '0111111001111110010011100100110110100001111000010000001100110011011000111101000111000100001110011000101101101010100110101110110', '0001010111001111111010000100000111000100001100010111101010001001001010101100001000100010111010001000101010101110101011101000101', '0001111011111011111011011110111111000001101101110001110010010110001101100111101000100010010110100011011111000100011100110010101', '1001011010001111100001000101011111000000110001011011101000101110001110101001101100000100100110010000100001010011011000000011101', '1111000011011101110010011110011111000110101101000010010111111010110110100111101010010010000010111000111011100101101010110111101', '10111010010110111110110101001

### Use sessions and advanced options

Explore sessions and advanced options to optimize circuit performance on QPUs.

In [8]:
import numpy as np
from qiskit.circuit.library import IQP
from qiskit.quantum_info import random_hermitian
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import Session, SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService

n_qubits = 127

service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=n_qubits
)

rng = np.random.default_rng(1234)
mat = np.real(random_hermitian(n_qubits, seed=rng))
circuit = IQP(mat)
circuit.measure_all()
mat = np.real(random_hermitian(n_qubits, seed=rng))
another_circuit = IQP(mat)
another_circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)
another_isa_circuit = pm.run(another_circuit)

with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([isa_circuit])
    another_job = sampler.run([another_isa_circuit])
    result = job.result()
    another_result = another_job.result()

# first job

print(
    f" > The first ten measurement results of job 1: {result[0].data.meas.get_bitstrings()[:10]}"
)

 > The first ten measurement results of job 1: ['1001000100011011011110100110001011110011011110010000111001001100100011000100000100010101110000000100100011000010010011000100001', '0001001100011101000100101100010001110100101111000000011101011001001100110000011001000100010010001110011100100000011000100001001', '1101111010011001010001001010001010011111011010000100101010010000000000000011000001100000100010100000100000100110011100101100010', '1001001101100001010101101100101111000110000010010000000001000001000000100010001000010001100100001000101001000000010110110101001', '1111001000011101111000010010100100101000011101011010010110010101001111100010010001000000100111100100110000100000000000001001011', '0010001100110001001001101001000100101110000101000000001110000100000000110000001100000100000001111010110011000001001000001111011', '0100101000001000010110100110101010110001110011010000001010001000001100010011011100010110101101111100001010100000010100010001000', '0001110000010000001000101110100001

In [9]:
# second job
print(
    " > The first ten measurement results of job 2:",
    another_result[0].data.meas.get_bitstrings()[:10],
)

 > The first ten measurement results of job 2: ['1110111101001011110110000010010111011100101110100000011101110010000101100110000000101001011010101001011100000100100100101000101', '0001101001110010011000010000011110001111011100000010100100110100001000010100010000000010001000011010100000100100001100000000110', '1101100000010111110100011011110010010001000100000011110000111110100001100101010001100011100000101010000110000010010110101001100', '0101110101011000110100000100000100100011110110000011011101011100000001111100100001000110011000101011001000111100001100100010000', '1100010100111010101100000011011010000111101100000011100001110100100000011110011101000010100000000001111010010000001000100000110', '1001000001100011001101000100111110111111000100110011111100101010010101010101000000100010101000001011000010000100001000000010011', '0111100000001011001110100110001110010000001100001110101000010110110010110111100010101000001100010011101010110110000110001000100', '0000010000010001101001000000001001

## Next steps

<Admonition type="tip" title="Recommendations">
    - [Specify advanced runtime options.](runtime-options-overview)
    - Practice with primitives by working through the [Cost function lesson](https://learning.quantum.ibm.com/course/variational-algorithm-design/cost-functions#primitives) in IBM Quantum Learning.
    - Learn how to transpile locally in the [Transpile](./transpile/) section.
    - Try the [Submit pre-transpiled circuits](https://learning.quantum.ibm.com/tutorial/submitting-user-transpiled-circuits-using-primitives) tutorial.
    - Read [Migrate to V2 primitives](/migration-guides/v2-primitives).
    - Understand the [Job limits](/guides/job-limits) when sending a job to an IBM&reg; QPU.
</Admonition>